In [96]:
#ライブラリをインポート
import csv
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [97]:
#マクロを定義
BINS = 10000  #ヒストグラムのビンの数
EPSILON = .00001


In [98]:
#KLダイバージェンス関数
def KLD(a, b):
    #a,bの最小値のより小さい方をmin_valueに
    min_value = min(min(a), min(b))
    #a,bの最大値のより大きい方をmax_valueに
    max_value = max(max(a), max(b))

    #a,bのヒストグラムを作成し、同じ数のビンで区切る
    a_hist, _ = np.histogram(a, bins=BINS, range=(min_value, max_value))
    b_hist, _ = np.histogram(b, bins=BINS, range=(min_value, max_value))

    #正規化する(確率分布に変換する、合計を1にする)ために全合計で割る
    a_hist = (a_hist + EPSILON) / a_hist.sum()
    b_hist = (b_hist + EPSILON) / b_hist.sum()
    #KLダイバージェンスの値を返す
    return np.sum([ai * np.log(ai / bi) for ai, bi in zip(a_hist, b_hist)])

In [99]:
#ファイルを読み込む
filename = "walk100Hz-0809-0900.csv"

In [100]:
#CSVファイルから各軸の加速度を読み込むための変数
accX = []
accY = []
accZ = []
Hz = 100

In [101]:
#ｘ軸,y軸,z軸の値を配列に格納
with open(filename) as f:
    reader = csv.reader(f)
    for row in reader:
        accX.append(float(row[1]))
        accY.append(float(row[2]))
        accZ.append(float(row[3]))

In [102]:
#各軸の加速度の平均を求める
aveX = sum(accX) / len(accX)
aveY = sum(accY) / len(accY)
aveZ = sum(accZ) / len(accZ)

In [103]:
#重力加速度の推定値=合成加速度の平均を求める
aveAcc = math.sqrt(aveX ** 2 + aveY ** 2 + aveZ ** 2)
print(aveAcc)

33680826.00137324


In [104]:
#各時刻の合成加速度を求める
acc = [math.sqrt(x ** 2 + y ** 2 + z ** 2) for x, y, z in zip(accX, accY, accZ)]

In [105]:
print(acc[:5])
len(acc)

[32443943.12708283, 32443953.18853604, 32443963.305498004, 32443973.30363897, 32443983.340506032]


247378

In [106]:
#各時刻の合成加速度から静止区間(重力加速度の推定値に近い値が5サンプル以上連続している区間)を除去する
counter = 0
n = 0
for i, stop in enumerate(acc):
    if stop < aveAcc * 1.1 and stop > aveAcc * 0.9:
        counter += 1
        if counter == 5:
            del acc[n:n+5]
            counter = 0
            i -= 5
    else:
        counter = 0
        n = i + 1

In [107]:
print(acc[:5])
len(acc)

[33680841.05041182, 33680851.050547235, 33680861.05157532, 33680871.04480135, 33680881.04623065]


123688

In [108]:
index = 0
accDif = []
for dif in acc[:-1]:
    accDif.append(math.fabs(acc[index + 1]*100000 - acc[index]*100000))
    index += 1

In [109]:
print(accDif[:5])
len(accDif)

[1000013.5415039062, 1000102.8081054688, 999322.6030273438, 1000142.9301757812, 1000431.4565429688]


123687

In [110]:
KLD(accDif, accDif)

0.0